In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

from pathlib import Path
import sys
import os
sys.path.append(os.path.abspath("..")) 
from scripts.get_paths import get_path
pd.set_option('display.max_columns', None)

In [2]:
paths = get_path()
df = pd.read_csv(paths.features_2 / "extracted_features.csv", index_col=False)

#### Classification of SAD vs Non-SAD with XGBoost - potential for clinical applications

In [3]:
GROUP_COLS = ["participant_id"]

# Define all candidate numeric features here once
FEATURES = {
    "pitch_pyafar": ["mean", "std"],
    "yaw_pyafar": ["mean", "std"],
    "roll_pyafar": ["mean", "std"],
    "gaze_pyafar": ["mean", "std"],
    "smile_pyafar": ["mean", "std"],
    
    "Occ_au_4": ["mean", "std"],
    "Occ_au_6": ["mean", "std"],
    "gaze_pyafar_pre": ["mean", "std"],
    "smile_pyafar_pre": ["mean", "std"],
    "nod_pyafar": ["mean", "std"],
}

# Choose which base features are active (easy toggle)
# all features extracted using pyafar and eye-contact-cnn are used currently 
ACTIVE_FEATURES = [
    "pitch_pyafar",
    "yaw_pyafar",
    "roll_pyafar",
    "gaze_pyafar_pre",
    #"gaze_pyafar",
    #"smile_pyafar",
    #"nod_pyafar",
    "Occ_au_4",
    "Occ_au_6",
    "smile_pyafar_pre"
    
]

N_SPLITS = 3
RANDOM_STATE = 32
# XGBoost params 
XGB_PARAMS = dict(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary:logistic",
    eval_metric="auc",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)

# Build agg spec only for active features
agg_spec = {feat: FEATURES[feat] for feat in ACTIVE_FEATURES}

# Aggregate
agg = (
    df.groupby(GROUP_COLS)
      .agg(agg_spec)
      .reset_index()
)

# Flatten columns
agg.columns = [
    "_".join(col).rstrip("_") if isinstance(col, tuple) else col
    for col in agg.columns
]

# Build the numeric feature column names automatically
num_features = []
for feat in ACTIVE_FEATURES:
    for stat in FEATURES[feat]:
        num_features.append(f"{feat}_{stat}")

# Safety checks
missing_cols = [c for c in num_features if c not in agg.columns]
if missing_cols:
    raise ValueError(
        "These expected aggregated columns are missing. "
        "Check ACTIVE_FEATURES / FEATURES / raw df columns:\n"
        + "\n".join(missing_cols)
    )

X_num = agg[num_features]

X = X_num

groups = agg["participant_id"]

# labels
y = agg["participant_id"].map(
    df.drop_duplicates("participant_id").set_index("participant_id")["NT"]
).astype(int).to_numpy()

xgb = XGBClassifier(**XGB_PARAMS)
logo = LeaveOneGroupOut()

seeds = range(0, 50)   # 50 runs with different model seeds
aucs = []

for s in seeds:
    xgb = XGBClassifier(
        n_estimators=300,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="binary:logistic",
        eval_metric="auc",
        random_state=s,      
        n_jobs=-1,
    )

    pipe = Pipeline([
        ("impute", SimpleImputer(strategy="median")),
        ("model", xgb),
    ])

    y_proba = cross_val_predict(
        pipe,
        X,
        y,
        groups=groups,
        cv=logo,
        method="predict_proba",
        n_jobs=-1,
    )[:, 1]

    aucs.append(roc_auc_score(y, y_proba))

aucs = np.array(aucs)
print("LOPO AUC over seeds:")
print("mean:", aucs.mean())
print("std :", aucs.std(ddof=1))
print("min :", aucs.min())
print("max :", aucs.max())


LOPO AUC over seeds:
mean: 0.7413492063492063
std : 0.015162465174329034
min : 0.7023809523809523
max : 0.7698412698412699
